In [1]:
# Device configuration
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [42]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import DataLoader
import os
import utils

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define directories
base_dir = '..\\misc_ds'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_df = utils.create_df(train_dir)
validation_df = utils.create_df(validation_dir)
test_df = utils.create_df(test_dir)

classes = [class_name for class_name in os.listdir(train_dir)]

# Create datasets and dataloaders
train_dataset = utils.CustomDataset(train_df, transform=utils.preprocess)
validation_dataset = utils.CustomDataset(validation_df, transform=utils.preprocess)
test_dataset = utils.CustomDataset(test_df, transform=utils.preprocess)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# Load pretrained MobileNetV2 model
mobilenet = models.mobilenet_v2(weights='MobileNet_V2_Weights.DEFAULT')

# Freeze convolutional layers
for param in mobilenet.parameters():
    param.requires_grad = False

# Modify the fully connected layer for the number of classes
num_ftrs = mobilenet.classifier[1].in_features
mobilenet.classifier[1] = nn.Linear(num_ftrs, len(classes))
mobilenet = mobilenet.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mobilenet.parameters(), lr=0.001)

# Training function
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(train_loader.dataset)

# Testing function
def test(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return running_loss / len(test_loader.dataset), correct / total

best_val_loss = float('inf')
best_model_state = None
patience = 3 
counter = 0

for epoch in range(1, 21):  # Change num_epochs to 20
    train_loss = train(mobilenet, train_loader, criterion, optimizer, device)
    val_loss, val_acc = test(mobilenet, validation_loader, criterion, device)
    print(f'Epoch [{epoch}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

    # Check for early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = mobilenet.state_dict()
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print(f'Early stopping at epoch {epoch} due to no improvement in validation loss.')
            break

# Save the best model
if best_model_state is not None:
    torch.save(mobilenet.state_dict(), 'best_model.pth')
    print('Best model saved.')

# Load the best model state
if best_model_state is not None:
    mobilenet.load_state_dict(best_model_state)
    print('Best model loaded.')

# Test the best model
test_loss, test_acc = test(mobilenet, test_loader, criterion, device)
print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')


Epoch [1], Train Loss: 1.3577, Val Loss: 0.9007, Val Acc: 0.8750
Epoch [2], Train Loss: 0.7587, Val Loss: 0.5672, Val Acc: 0.8833
Epoch [3], Train Loss: 0.5617, Val Loss: 0.4443, Val Acc: 0.8833
Epoch [4], Train Loss: 0.4518, Val Loss: 0.3747, Val Acc: 0.8917
Epoch [5], Train Loss: 0.3930, Val Loss: 0.3395, Val Acc: 0.9000
Epoch [6], Train Loss: 0.3339, Val Loss: 0.3080, Val Acc: 0.8917
Epoch [7], Train Loss: 0.3039, Val Loss: 0.2908, Val Acc: 0.9000
Epoch [8], Train Loss: 0.2608, Val Loss: 0.2774, Val Acc: 0.9000
Epoch [9], Train Loss: 0.2573, Val Loss: 0.2641, Val Acc: 0.8917
Epoch [10], Train Loss: 0.2306, Val Loss: 0.2581, Val Acc: 0.8917
Epoch [11], Train Loss: 0.2115, Val Loss: 0.2467, Val Acc: 0.9083
Epoch [12], Train Loss: 0.2048, Val Loss: 0.2455, Val Acc: 0.9000
Epoch [13], Train Loss: 0.1883, Val Loss: 0.2370, Val Acc: 0.9167
Epoch [14], Train Loss: 0.1902, Val Loss: 0.2380, Val Acc: 0.9083
Epoch [15], Train Loss: 0.1679, Val Loss: 0.2293, Val Acc: 0.9167
Epoch [16], Train L

In [43]:
#exporting the model

torch_model = mobilenet.cpu()
torch_input = torch.randn(1, 3, 224, 224)
torch.onnx.export(torch_model,              # model being run
                  torch_input,                         # model input (or a tuple for multiple inputs)
                  "other_classifier.onnx",     # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

In [44]:
import onnxruntime
import numpy as np

# Load the exported ONNX model
ort_session = onnxruntime.InferenceSession("other_classifier.onnx")

# Initialize lists to store predictions from PyTorch and ONNX Runtime
pytorch_predictions = []
onnx_predictions = []

# Test the model with each batch in the test dataset
for batch in test_loader:
    x, y = batch

    # Compute PyTorch output prediction
    pytorch_output = mobilenet(x).detach().cpu().numpy()

    # Compute ONNX Runtime output prediction
    ort_inputs = {ort_session.get_inputs()[0].name: x.numpy()}  # Convert to numpy array
    ort_outs = ort_session.run(None, ort_inputs)
    onnx_output = ort_outs[0]

    # Append predictions to lists
    pytorch_predictions.append(pytorch_output)
    onnx_predictions.append(onnx_output)

# Concatenate lists to numpy arrays
pytorch_predictions = np.concatenate(pytorch_predictions)
onnx_predictions = np.concatenate(onnx_predictions)

# Compare PyTorch and ONNX Runtime results
np.testing.assert_allclose(pytorch_predictions, onnx_predictions, rtol=1e-03, atol=1e-05)

print("Model has been tested against the test dataset using both PyTorch and ONNX Runtime, and the results are consistent.")

Model has been tested against the test dataset using both PyTorch and ONNX Runtime, and the results are consistent.


In [85]:
from PIL import Image

# Load the exported ONNX model
ort_session = onnxruntime.InferenceSession("other_classifier.onnx")

# Load and preprocess a single image
image_test_path_dir = '..\\misc_ds\\validation\\egg'

for filename in os.listdir(image_test_path_dir):
    # Load and preprocess the image
    img_path = os.path.join(image_test_path_dir, filename)
    img = Image.open(img_path)
    transformed_img = utils.preprocess(img)  # Assuming utils.preprocess performs the necessary transformations

    # Add a batch dimension to the image tensor
    transformed_img = transformed_img.unsqueeze(0)  # Add batch dimension

    # Compute ONNX Runtime output prediction
    ort_inputs = {ort_session.get_inputs()[0].name: transformed_img.numpy()}  # Convert to numpy array
    ort_outs = ort_session.run(None, ort_inputs)
    onnx_output = ort_outs[0]

    # Get the predicted class index
    max_index = np.argmax(onnx_output)
    print("ONNX Runtime output prediction:", classes[max_index])

ONNX Runtime output prediction: egg
ONNX Runtime output prediction: egg
ONNX Runtime output prediction: egg
ONNX Runtime output prediction: egg
ONNX Runtime output prediction: egg
ONNX Runtime output prediction: egg
ONNX Runtime output prediction: egg
ONNX Runtime output prediction: egg
ONNX Runtime output prediction: egg
ONNX Runtime output prediction: egg
ONNX Runtime output prediction: egg
ONNX Runtime output prediction: egg
ONNX Runtime output prediction: egg
ONNX Runtime output prediction: egg
ONNX Runtime output prediction: egg
ONNX Runtime output prediction: egg
ONNX Runtime output prediction: egg
ONNX Runtime output prediction: egg
ONNX Runtime output prediction: egg
ONNX Runtime output prediction: egg
